# Test Loading GeoMAD items with ODC-stac 

In [ ]:
import pystac
import matplotlib.pyplot as plt
import numpy as np
from datacube import Datacube
from odc.stac import load

In [ ]:
BASE = "https://piksel-staging-public-data.s3.ap-southeast-3.amazonaws.com"
COLLECTION = "gm_s2_testd"
VERSION = "0.0.1"
PERIOD = "2024--P1Y"

TARGET_TILES = {
    (211, -1),
    (211, -2),
    (212, -1),
    (212, -2),
    (256, -8),
    (256, -7),
}

In [ ]:
def fmt_y(y: int) -> str:
    s = f"{y:+03d}"
    return s.replace("+", "")

def stac_item_url(x: int, y: int, period: str = PERIOD) -> str:
    yy = fmt_y(y)
    return (
        f"{BASE}/{COLLECTION}/{VERSION}/"
        f"x{x:03d}/y{yy}/"
        f"{period}/gm_s2_annual_x{x:03d}y{yy}_{period}.stac-item.json"
    )

stac_item_url(211, -1)

In [ ]:
items = []
for x, y in sorted(TARGET_TILES):
    url = stac_item_url(x, y)
    item = pystac.Item.from_file(url)
    item.properties["x"] = x
    item.properties["y"] = y
    items.append(item)

print("Loaded:", len(items))
print("Assets:", list(items[0].assets.keys()))

In [ ]:
# inspect stac-item
items[0]

In [ ]:
# load data
data = load(items, chunks={"x": 2048, "y": 2048, "time": 1})

# Inspect the geobox
data.odc.geobox

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(12, 5), constrained_layout=True)

for ax, item in zip(axs.flat, items):
    ds = load([item], bands=["red", "green", "blue"], chunks={"x": 5000, "y": 5000}).isel(time=0)

    rgb = ds[["red", "green", "blue"]].to_array("band").transpose("y", "x", "band")
    rgb.plot.imshow(ax=ax, vmin=0, vmax=3000, add_colorbar=False)

    ax.set_title(item.properties.get("odc:region_code", item.id))
    ax.set_axis_off()

plt.show()

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(12, 5), constrained_layout=True)

for ax, item in zip(axs.flat, items):
    ds = load([item], bands=["SMAD","COUNT"], chunks={"x": 5000, "y": 5000}).isel(time=0)
    smad = ds["SMAD"].where(ds["COUNT"] > 0)

    smad.plot.imshow(ax=ax, cmap="magma", vmin=0, vmax=0.03, add_colorbar=False)
    ax.set_title(item.properties.get("odc:region_code", item.id))
    ax.set_axis_off()

plt.show()